In [1]:
import pandas as pd
import tensorflow as tf

In [2]:
# Input layer
input_layer = tf.keras.layers.Input(shape=(256,))

# Hidden layer with ReLU activation
hidden_layer = tf.keras.layers.Dense(32, activation='relu')(input_layer)
hidden_layer = tf.keras.layers.Dense(32, activation='relu')(hidden_layer)

# Output layer with sigmoid activation for binary classification
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(hidden_layer)

# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['auc'])

# You can now use this 'model' object for training, evaluation, and prediction.
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense (Dense)               (None, 32)                8224      
                                                                 
 dense_1 (Dense)             (None, 32)                1056      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 9313 (36.38 KB)
Trainable params: 9313 (36.38 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
tinit_train = pd.read_csv('tinit_train.csv')
tinit_test = pd.read_csv('tinit_test.csv')
wavefake_train = pd.read_csv('wavefake_train.csv')
wavefake_test = pd.read_csv('wavefake_test.csv')

In [10]:
train_x = pd.concat([
    pd.read_csv('tinit_train_x.csv', header=None),
    pd.read_csv('wavefake_train_x.csv', header=None)
])

train_y = pd.concat([tinit_train['fake'], wavefake_train['fake']])

test_x = pd.concat([
    pd.read_csv('tinit_test_x.csv', header=None),
    pd.read_csv('wavefake_test_x.csv', header=None)
])
test_y = pd.concat([tinit_test['fake'], wavefake_test['fake']])

In [11]:
train_x.shape, test_x.shape

((105320, 256), (35246, 256))

In [12]:
train_y.shape, test_y.shape

((105320,), (35246,))

In [13]:
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight

neg, pos = np.bincount(train_y)
total = neg + pos
# Scaling by total/2 helps keep the loss to a similar magnitude.
# The sum of the weights of all examples stays the same.
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}
print(class_weight)

# Create the early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model with the specified metrics
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=[tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])


# Train the model with class weights and early stopping
history = model.fit(train_x, train_y, epochs=300, batch_size=128 , class_weight=class_weight, validation_data=(test_x, test_y), callbacks=[early_stopping])


{0: 11.398268398268398, 1: 0.5229394240317775}
Epoch 1/300
823/823 [==============================] - 2s 2ms/step - loss: 0.2146 - auc: 0.9881 - precision: 0.9991 - recall: 0.9734 - val_loss: 0.0039 - val_auc: 0.9999 - val_precision: 0.9999 - val_recall: 0.9990
Epoch 2/300
823/823 [==============================] - 1s 1ms/step - loss: 0.0111 - auc: 0.9995 - precision: 0.9999 - recall: 0.9967 - val_loss: 0.0043 - val_auc: 0.9999 - val_precision: 1.0000 - val_recall: 0.9987
Epoch 3/300
823/823 [==============================] - 1s 1ms/step - loss: 0.0108 - auc: 0.9995 - precision: 0.9999 - recall: 0.9970 - val_loss: 0.0023 - val_auc: 0.9999 - val_precision: 1.0000 - val_recall: 0.9995
Epoch 4/300
823/823 [==============================] - 1s 1ms/step - loss: 0.0136 - auc: 0.9991 - precision: 0.9999 - recall: 0.9972 - val_loss: 0.0011 - val_auc: 1.0000 - val_precision: 0.9999 - val_recall: 0.9999
Epoch 5/300
823/823 [==============================] - 1s 1ms/step - loss: 0.0342 - auc: 0.99

In [14]:
model.save('fake_classifier')

INFO:tensorflow:Assets written to: fake_classifier\assets


INFO:tensorflow:Assets written to: fake_classifier\assets
